In [1]:
import configparser
from pathlib import Path
from datetime import datetime
from os import PathLike

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 0)

In [2]:
def read_config(fn):
    # reads in an ini config file, including any blank lines

    # read in file, add "Null" to any empty inputs
    fn = Path(fn)
    with open(fn, 'r') as f:
        lines = f.readlines()
    for i, _ in enumerate(lines):
        if lines[i][-2:] == '=\n':
            lines[i] = lines[i][:-1] + 'Null' + lines[i][-1]
    complete_fn = fn.parent / (fn.stem + '_complete.eddypro')
    with open(complete_fn, 'w') as f:
        for ln in lines:
            f.write(ln)

    ini = configparser.ConfigParser()
    ini.read(complete_fn)
    return ini

def read_config_as_dataframe(fn):
    lines = []
    ini = read_config(fn)
    for section in ini.sections():
        for option, value, in ini[section].items():
            lines.append([section, option, value])
    df = pd.DataFrame(lines, columns=['Section', 'Option', 'Value'])
    df = df.sort_values(['Section', 'Option'])
    df['Name'] = fn.stem
    return df

def compare_configs(df1: pd.DataFrame, df2: pd.DataFrame):
    # compare differences between two config dataframe
    df1_new = df1.loc[df1['Value'] != df2['Value'], ['Section', 'Option', 'Value']]
    df2_new = df2.loc[df1['Value'] != df2['Value'], ['Section', 'Option', 'Value']]
    name1 = df1['Name'].values[0]
    name2 = df2['Name'].values[0]
    df_compare = (
        df1_new
        .merge(df2_new, on=['Section', 'Option'], suffixes=['_'+name1, '_'+name2])
        .sort_values(['Section', 'Option'])
    )
    return df_compare

In [3]:
environment = Path('/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro')

# When we hit buttons in EddyPro, what settings change in the .ini file?


## Set up a template
First, we create a reference .eddypro file. This file will have all the default processing settings.

Note that EddyPro will leave some options blank

In [21]:
base = read_config_as_dataframe(environment / 'ini/base.eddypro')
base.head()

,Section,Option,Value,Name
0,FluxCorrection_SpectralAnalysis_General,add_sonic_lptf,1,base
1,FluxCorrection_SpectralAnalysis_General,ex_dir,Null,base
2,FluxCorrection_SpectralAnalysis_General,ex_file,Null,base
3,FluxCorrection_SpectralAnalysis_General,horst_lens,2,base
4,FluxCorrection_SpectralAnalysis_General,sa_bin_spectra,Null,base


## Basic Settings
### Changing dates
change **Basic-Settings/Start** and **Basic-Settings/End**

In [5]:
change_dates = read_config_as_dataframe(environment / 'ini/change_dates.eddypro')
compare_configs(base, change_dates)

,Section,Option,Value_base,Value_change_dates
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/change_dates.eddypro
1,Project,files_found,0,1386
2,Project,last_change_date,2023-07-28T10:38:41,2023-07-26T11:11:27
3,Project,pr_end_date,2020-07-22,2020-06-26
4,Project,pr_end_time,00:00,04:30
5,Project,pr_start_date,2020-06-21,2020-06-23
6,Project,pr_start_time,00:00,02:30
7,Project,project_id,base,change_dates
8,RawProcess_Settings,out_bin_sp,1,0
9,RawProcess_Settings,tlag_meth,4,2


other than trivial changes and housekeeping changes like file_name, last_change_date, and project_id, the only changed settings are the start and end times and dates, as expected:

These are pretty self-explanatory:
* Project/pr_end_time
* Project/pr_start_time
* Project/pr_end_date
* Project/pr_start_date

### Missing Samples Allowance and Flux Averaging Interval
Change **Basic-Settings/Missing-Samples-Allowance** and **Basic-Settings/Flux-Averaging-Interval**

In [6]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/missing-avgint.eddypro')
)


,Section,Option,Value_base,Value_missing-avgint
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/missing-avgint.eddypro
1,Project,files_found,0,1386
2,Project,last_change_date,2023-07-28T10:38:41,2023-07-26T13:36:21
3,Project,project_id,base,missing-avgint
4,RawProcess_Settings,avrg_len,30,0
5,RawProcess_Settings,max_lack,10,0
6,RawProcess_Settings,out_bin_sp,1,0
7,RawProcess_Settings,tlag_meth,4,2
8,RawProcess_TiltCorrection_Settings,pf_end_time,00:00,23:30


Changes to INI file:
* RawProcess_Settings/avrg_len: averaging interval 
* RawProcess_Settings/max_lack: maximum % missing values

### North Reference
Change **Basic-Settings/North-Reference** to Geographic North, but don't change 

In [7]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/northref.eddypro')
)

,Section,Option,Value_base,Value_northref
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/northref.eddypro
1,Project,files_found,0,1386
2,Project,last_change_date,2023-07-28T10:38:41,2023-07-26T11:37:33
3,RawProcess_General,dec_date,2023-12-31,2020-07-22
4,RawProcess_General,mag_dec,0,8.666667
5,RawProcess_General,use_geo_north,0,1
6,RawProcess_Settings,out_bin_sp,1,0
7,RawProcess_Settings,tlag_meth,4,2


* RawProcess_General/dec_date (yyyy-mm-dd)
* RawProcess_General/mag_dec (decimal declination)
* RawProcess_General/use_geo_north (0 -> 1)
* TiltCorrection_Settings/pf_end_time (23:30 -> 00:00??, ignoring for now)

### Wind speed measurement offsets
Change **Advanced-Settings/Wind-Speed-Measurement-Offsets**

In [8]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/windspeedoffsets.eddypro')
)

,Section,Option,Value_base,Value_windspeedoffsets
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/windspeedoffsets.eddypro
1,Project,files_found,0,1386
2,Project,last_change_date,2023-07-28T10:38:41,2023-07-26T14:07:54
3,RawProcess_Settings,out_bin_sp,1,0
4,RawProcess_Settings,tlag_meth,4,2
5,RawProcess_Settings,u_offset,0,1.0999999999999999
6,RawProcess_Settings,v_offset,0,-1.0999999999999999
7,RawProcess_Settings,w_offset,0,-0.30000000000000004
8,RawProcess_TiltCorrection_Settings,pf_end_time,00:00,23:30


### Axis rotations for tilt correction
options are:

* Untick **Advanced-Settings/Processing-Options/Axis-Rotations-For-Tilt-Correction**
* 

In [9]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/norot.eddypro')
)

,Section,Option,Value_base,Value_norot
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/norot.eddypro
1,Project,files_found,0,1386
2,Project,last_change_date,2023-07-28T10:38:41,2023-07-26T14:14:00
3,Project,project_id,base,norot
4,RawProcess_Settings,out_bin_sp,1,0
5,RawProcess_Settings,rot_meth,1,0
6,RawProcess_Settings,tlag_meth,4,2
7,RawProcess_TiltCorrection_Settings,pf_end_time,00:00,23:30


In [10]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/tr.eddypro')
)

,Section,Option,Value_base,Value_tr
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/tr.eddypro
1,Project,files_found,0,1386
2,Project,last_change_date,2023-07-28T10:38:41,2023-07-26T14:24:22
3,Project,project_id,base,tr
4,RawProcess_Settings,out_bin_sp,1,0
5,RawProcess_Settings,rot_meth,1,3
6,RawProcess_Settings,tlag_meth,4,2
7,RawProcess_TiltCorrection_Settings,pf_end_time,00:00,23:30
8,RawProcess_TiltCorrection_Settings,pf_mode,1,0


In [11]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/pfnvb_with_file.eddypro')
)

,Section,Option,Value_base,Value_pfnvb_with_file
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/pfnvb_with_file.eddypro
1,Project,files_found,0,1386
2,Project,last_change_date,2023-07-28T10:38:41,2023-07-26T14:44:43
3,Project,project_id,base,pfnvb_with_file
4,RawProcess_Settings,out_bin_sp,1,0
5,RawProcess_Settings,rot_meth,1,4
6,RawProcess_Settings,tlag_meth,4,2
7,RawProcess_TiltCorrection_Settings,pf_end_date,2020-07-22,2020-07-23
8,RawProcess_TiltCorrection_Settings,pf_end_time,00:00,20:30
9,RawProcess_TiltCorrection_Settings,pf_start_date,2020-06-21,2020-03-03


Unticking tilt corrections: rot_meth=0

Selecting triple rotations: rot_meth=2

Selecting double rotations: rot_meth=1

Selecting planar fit with a provided file: rot_meth=3, RawProcess_TiltCorrection_Settings/pf_mode = 0, pf_file=filepath

Selecting planar fit (no v bias) with a provided file: rot_meth=4, RawProcess_TiltCorrection_Settings/pf_mode = 0, pf_file=filepath

When providing planar fit settings manually, change the following in RawProcess_TiltCorrection_Settings

* pf_start/end_date/time
* pf_subset: 1???
* pf_mode: 1 for manual, 0 for file
* pf_u_min: minimum mean horizontal windspeed [0, 10]
* pf_w_max: maximum mean vertical windspeed (0, 10]
* pf_min_num_per_sec: minimum elements per sector [0, 9999]
* pf_fix: what to do if calculations fail for a sector (closest clockwise 0, closest ccw 1, dr 2)
* pf_north_offset: north offset for minimum azimuth of first sector
* sectors: n goes from 1 to max of 10
  * pf_sector_n_exclude: 1 to ignore for PF, 0 to use for PF
  * pf_sector_n_width: degrees, from 1 to 359

### Extracting Turbulent Fluctuations
This one is in **Advanced-Settings/Processing/Turbulent-fluctuations** and is pretty simple:

RawProcess_Settings/detrend_meth: 0 (block avg), 1 (linear detrending), 2 (running mean), 3 (exponential running mean)

RawProcess_Settings/timeconst: 0 gives same as flux averaging interval, otherwise units are seconds.

### Time lags compensation


In [13]:
read_config_as_dataframe(environment / 'ini/tlags_complete.eddypro').query('Section == "RawProcess_TimelagOptimization_Settings"').sort_values('Option')

,Section,Option,Value,Name
350,RawProcess_TimelagOptimization_Settings,to_ch4_max_lag,-1000.1,tlags_complete
351,RawProcess_TimelagOptimization_Settings,to_ch4_min_flux,0.200,tlags_complete
352,RawProcess_TimelagOptimization_Settings,to_ch4_min_lag,-1000.1,tlags_complete
353,RawProcess_TimelagOptimization_Settings,to_co2_max_lag,-1000.1,tlags_complete
354,RawProcess_TimelagOptimization_Settings,to_co2_min_flux,2.000,tlags_complete
355,RawProcess_TimelagOptimization_Settings,to_co2_min_lag,-1000.1,tlags_complete
356,RawProcess_TimelagOptimization_Settings,to_end_date,2020-07-22,tlags_complete
357,RawProcess_TimelagOptimization_Settings,to_end_time,00:00,tlags_complete
358,RawProcess_TimelagOptimization_Settings,to_file,Null,tlags_complete
359,RawProcess_TimelagOptimization_Settings,to_gas4_max_lag,-1000.1,tlags_complete


In [14]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/tlags.eddypro')
).sort_values('Option')

,Section,Option,Value_base,Value_tlags
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/tlags.eddypro
1,Project,files_found,0,1386
2,Project,last_change_date,2023-07-28T10:38:41,2023-07-26T19:05:49
4,RawProcess_TiltCorrection_Settings,pf_end_time,00:00,23:30
3,Project,project_id,base,tlags
5,RawProcess_TimelagOptimization_Settings,to_ch4_max_lag,-1000.1,-999.3
6,RawProcess_TimelagOptimization_Settings,to_ch4_min_flux,0.200,0.202
7,RawProcess_TimelagOptimization_Settings,to_ch4_min_lag,-1000.1,-999.4
8,RawProcess_TimelagOptimization_Settings,to_co2_max_lag,-1000.1,1000.0
9,RawProcess_TimelagOptimization_Settings,to_co2_min_flux,2.000,1.998


RawProcess_Settings/tlag_meth:
* No compensation: tlag_meth = 0
* Constant: tlag_meth = 1, as specified in metadata
* Covariance maximization with default: 2
* Naive maximization: 3
* Automatic time lag optimization: provide a file or manually modify settings: 4
    * provide a file:
      * RawProcess_TimeLagOptimization_Settings to_file = path to timelag_opt output file
      * RawProcess_TimeLagOptimization_Settings to_mode = 0
      * RawProcess_TimeLagOptimization_Settings to_subset = 0
    * manually input settings:
      * RawProcess_TimeLagOptimization_Settings to_subset = 0 or 1?????
      * RawProcess_TimeLagOptimization_Settings to_mode = 1
      * to_co2/ch4/h2o/gas4_max/min_lag: -1000s to +1000s, max > min
      * to_start/end_date/time
      * etc...(FINISH)



In [15]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/outs.eddypro')
)

,Section,Option,Value_base,Value_outs
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/outs.eddypro
1,Project,last_change_date,2023-07-28T10:38:41,2023-07-28T14:17:50


### Compensate Density Fluctuations

In [28]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/densfluct.eddypro')
)

,Section,Option,Value_base,Value_densfluct
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/densfluct.eddypro
1,Project,files_found,1386,0
2,Project,last_change_date,2023-08-08T18:01:11,2023-08-08T18:05:14
3,RawProcess_Settings,bu_corr,0,1
4,RawProcess_Settings,bu_multi,0,1
5,RawProcess_Settings,m_day_bot1,2.80000000,2.81000000
6,RawProcess_Settings,m_day_bot2,-0.06810000,1.00000000
7,RawProcess_Settings,m_day_bot3,0.00210000,1.00000000
8,RawProcess_Settings,m_day_bot4,-0.33400000,1.00000000
9,RawProcess_Settings,m_day_spar1,0.30000000,1.00000000


Disabling Compensate density fluctuations:
* Project/wpl_meth 1->0

Enable instr. sensible head components
* RawProcess_Settings/bu_corr 0->1
* Simple linear vs multiple reg: Rawprocess_settings/bu_multi 0->1
* Changing offsets (RawProcess_Settings)
  * l_day/night_bot/top/spar_gain/offset for simple linear
  * m_day/night_bot/top/spar_1/2/3/4 for MLR

